In [9]:
from datetime import datetime
from typing import Type
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain.utilities.wikipedia import WikipediaAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.document_loaders.web_base import WebBaseLoader
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

class SearchQuery(BaseModel):
    query: str = Field(description="The search query. Example: Research about the XZ backdoor")

class LoadURL(BaseModel):
    url: str = Field(description="The URL to load. Example: https://en.wikipedia.org/wiki/Backdoor_(computing)")

class FileSaveArgs(BaseModel):
    text: str = Field(description="The text to save to a file.")
    file_path: str = Field(description="Path of the file to save the text to.")

class WikipediaSearch(BaseTool):
    name = "WikipediaSearch"
    description = "Finds websites for a given query."
    args_schema: Type[SearchQuery] = SearchQuery

    def _run(self, query):
        wrapper = WikipediaAPIWrapper()
        return wrapper.run(query)

class DuckDuckGoSearch(BaseTool):
    name = "DuckDuckGoTool"
    description = "Finds websites for a given query."
    args_schema: Type[SearchQuery] = SearchQuery

    def _run(self, query):
        wrapper = DuckDuckGoSearchAPIWrapper()
        return wrapper.run(query)

class WebsiteLoader(BaseTool):
    name = "WebsiteLoader"
    description = "Loads the website for a given URL."
    args_schema: Type[LoadURL] = LoadURL

    def _run(self, url):
        loader = WebBaseLoader([url])
        return loader.load()

class TextSaver(BaseTool):
    name = "TextSaver"
    description = "Saves the text to a file."
    args_schema: Type[FileSaveArgs] = FileSaveArgs

    def _run(self, text, file_path):
        current_time = datetime.now().strftime("%Y-%m-%d_jhun")
        with open(f"{current_time}_{file_path}", "w", encoding="utf-8") as file:
            file.write(text)
        return f"Text saved to {current_time}_{file_path}"

def agent_invoke(input):
    agent = initialize_agent(
        llm=llm,
        verbose=True,
        agent=AgentType.OPENAI_FUNCTIONS,
        handle_parsing_errors=True,
        tools=[
            WikipediaSearch(),
            DuckDuckGoSearch(),
            WebsiteLoader(),
            TextSaver(),
        ],
    )

    prompt = PromptTemplate.from_template(
        """    
        1. Search for the query,
        2. If there's a list of website URLs in the search results, extract the content of each website as text, and
        3. Save it to a txt file.
        Query: {query}    
        """,
    )

    chain = {"query": RunnablePassthrough()} | prompt | agent
    chain.invoke({"input": input})

query = "Research about the XZ backdoor"
agent_invoke(query)




> Entering new AgentExecutor chain...

Invoking: `WikipediaSearch` with `{'query': 'Research about the XZ backdoor'}`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been assigned a CVSS score of 10.0, the highest possible score.



Page: Supply chain attack
Summary: A supply chain attack is a cyber-attack that seeks to d